In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os

In [3]:
import os
import torch
from PIL import Image
from torchvision import transforms

# Paths
input_base = "../data/raw/images/Celeb_V2"
output_base = "../data/processed/Celeb_V2"
os.makedirs(output_base, exist_ok=True)

splits = ["Train", "Val", "Test"]
for split in splits:
    for label in ["real", "fake"]:
        os.makedirs(os.path.join(output_base, split, label), exist_ok=True)


In [4]:
img_size = 224

preprocess_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),   # Resize to 224x224
    transforms.ToTensor(),                     # Convert to tensor
    transforms.Normalize([0.485, 0.456, 0.406],  # ImageNet mean
                         [0.229, 0.224, 0.225])  # ImageNet std
])


In [5]:
# Example: load one image and apply preprocessing
img_path = "../data/raw/images/Celeb_V2/Train/real/00000_face_1.jpg"  # adjust path
image = Image.open(img_path).convert("RGB")  # ensure RGB
image_tensor = preprocess_transform(image)

print("Image tensor shape:", image_tensor.shape)
print("Tensor min/max:", image_tensor.min().item(), image_tensor.max().item())


Image tensor shape: torch.Size([3, 224, 224])
Tensor min/max: -2.1179039478302 2.5354249477386475


In [24]:
import os
from PIL import Image
import torch
from torchvision import transforms

# Example preprocess transform (you can modify if needed)
preprocess_transform = transforms.Compose([
    transforms.Resize((224, 224)),   # resize all images
    transforms.ToTensor(),           # convert to tensor
])

# Convert back to image for saving
to_pil = transforms.ToPILImage()

# Paths
input_base = "../data/raw/images/Celeb_V2"
output_base = "../data/processed_fixed/Celeb_V2"
splits = ["Train", "Val", "Test"]

for split in splits:
    for label in ["real", "fake"]:
        in_folder = os.path.join(input_base, split, label)
        out_folder = os.path.join(output_base, split, label)
        os.makedirs(out_folder, exist_ok=True)

        for img_name in os.listdir(in_folder):
            try:
                img_path = os.path.join(in_folder, img_name)
                img = Image.open(img_path).convert("RGB")

                # Apply transforms
                img_tensor = preprocess_transform(img)

                # Save as proper .jpg
                save_path = os.path.join(out_folder, img_name.split('.')[0] + ".jpg")
                to_pil(img_tensor).save(save_path)

            except Exception as e:
                print(f"Skipping {img_path}: {e}")


In [3]:

class DeepfakeDataset(Dataset):
    def __init__(self, data_dir, split, transform=None):
        """
        Args:
            data_dir (str): Base directory containing processed images (e.g., '../data/processed/Celeb_V2').
            split (str): 'Train', 'Val', or 'Test'.
            transform (callable, optional): Transform to apply to images.
        """
        self.data_dir = os.path.join(data_dir, split)
        self.transform = transform or transforms.ToTensor()
        self.images = []
        self.labels = []
        self.label_map = {'real': 0, 'fake': 1}

        # Collect image paths and labels
        for label in ['real', 'fake']:
            folder = os.path.join(self.data_dir, label)
            for img_name in os.listdir(folder):
                if img_name.endswith('.jpg'):
                    self.images.append(os.path.join(folder, img_name))
                    self.labels.append(self.label_map[label])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        label = self.labels[idx]
        return img, label


In [5]:
# Parameters
batch_size = 32
num_workers = 0   # use 0 for Jupyter/Windows to avoid freezing
data_dir = "../data/processed_fixed/Celeb_V2"

# Example transforms for training (resize + normalize for pretrained models)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# Create datasets
train_dataset = DeepfakeDataset(data_dir, "Train", transform=transform)
val_dataset   = DeepfakeDataset(data_dir, "Val", transform=transform)
test_dataset  = DeepfakeDataset(data_dir, "Test", transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                          num_workers=num_workers, pin_memory=torch.cuda.is_available())
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                          num_workers=num_workers, pin_memory=torch.cuda.is_available())
test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                          num_workers=num_workers, pin_memory=torch.cuda.is_available())

# Quick test
for images, labels in train_loader:
    print(f"Image batch shape: {images.shape}")  # [B, 3, 224, 224]
    print(f"Label batch shape: {labels.shape}")  # [B]
    break


Image batch shape: torch.Size([32, 3, 224, 224])
Label batch shape: torch.Size([32])


In [29]:
print("Train samples:", len(train_dataset))
print("Val samples:", len(val_dataset))
print("Test samples:", len(test_dataset))


Train samples: 80824
Val samples: 10104
Test samples: 10103


In [6]:
# MCNN Model
class DeepfakeCNN(nn.Module):
    def __init__(self, num_classes=2, input_size=224):
        super(DeepfakeCNN, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)

        # Compute flattened size dynamically
        with torch.no_grad():
            dummy = torch.zeros(1, 3, input_size, input_size)
            x = self.pool(F.relu(self.conv1(dummy)))
            x = self.pool(F.relu(self.conv2(x)))
            x = self.pool(F.relu(self.conv3(x)))
            self.flattened_size = x.numel()

        # Fully connected layers
        self.fc1 = nn.Linear(self.flattened_size, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = torch.flatten(x, 1)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

In [45]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torchvision.models import efficientnet_b0

# # Define the model
# class DeepfakeClassifier(nn.Module):
#     def __init__(self, num_classes=2, pretrained=True):
#         super(DeepfakeClassifier, self).__init__()
#         self.model = efficientnet_b0(pretrained=pretrained)
#         in_features = self.model.classifier[1].in_features
#         self.model.classifier = nn.Sequential(
#             nn.Dropout(p=0.2),
#             nn.Linear(in_features, num_classes)
#         )

#     def forward(self, x):
#         return self.model(x)

In [9]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, device):
    best_val_acc = 0.0
    for epoch in range(num_epochs):
        # Training
        model.train()
        train_loss, train_correct, train_total = 0.0, 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
        train_loss /= train_total
        train_acc = 100 * train_correct / train_total

        # Validation
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * images.size(0)
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        val_loss /= val_total
        val_acc = 100 * val_correct / val_total

        # Scheduler step
        scheduler.step()

        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            os.makedirs("models", exist_ok=True)
            torch.save(model.state_dict(), 'models/best_deepfake_model.pth')

        print(f"Epoch {epoch+1}/{num_epochs}: Train Loss={train_loss:.4f}, Train Acc={train_acc:.2f}%, Val Loss={val_loss:.4f}, Val Acc={val_acc:.2f}%")



In [11]:
def main():
    # Parameters
    data_dir = "../data/processed_fixed/Celeb_V2"
    batch_size = 8     
    num_epochs = 10
    learning_rate = 1e-4
    input_size = 224     # match preprocessing

    # Device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Transforms
    transform = transforms.Compose([
        transforms.Resize((input_size, input_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

    # Datasets & DataLoaders
    train_dataset = DeepfakeDataset(data_dir, "Train", transform=transform)
    val_dataset   = DeepfakeDataset(data_dir, "Val", transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=torch.cuda.is_available())
    val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=torch.cuda.is_available())

    print(f"Train samples: {len(train_dataset)}, Val samples: {len(val_dataset)}")

    # Model, loss, optimizer, scheduler
    model = DeepfakeCNN(num_classes=2, input_size=input_size).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

    # Train
    train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, device)

if __name__ == "__main__":
    main()

Using device: cpu
Train samples: 80824, Val samples: 10104


KeyboardInterrupt: 